In [30]:
import pandas as pd
import datetime
from datetime import date, time, timedelta
import time
import warnings

warnings.filterwarnings("ignore")

In [31]:
# Global variables

BATTERY_POWER = 300
BATTERY_CAP = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991

In [32]:
def raw_power(charge_forecast, discharge_forecast, opening_cap):
    '''Takes in the forecasted battery behaviour and opening capacity, returns the amount of raw power.'''
    
    if charge_forecast == 1 and discharge_forecast == 0:
        return -min(BATTERY_POWER, (BATTERY_CAP - opening_cap)/(CHARGE_EFF/100)*2)
    elif charge_forecast == 0 and discharge_forecast == 1:
        return min(BATTERY_POWER, opening_cap*2)
    else:
        return 0

In [33]:
def market_dispatch(raw_power):
    '''Takes in the raw power, returns the power for market dispatch.'''
    
    if raw_power < 0:
        return (raw_power/2)
    elif raw_power > 0:
        return (raw_power/2)*DISCHARGE_EFF/100
    else:
        return 0

In [34]:
def market_revenue(market_dispatch,spot_price):
    '''Takes in the power for market dispatch and spot price, returns the market revenue generated.'''
    
    if market_dispatch < 0:
        return market_dispatch*spot_price*(1/MLF)
    elif market_dispatch > 0:
        return market_dispatch*spot_price*MLF
    else:
        return 0

In [35]:
def closing_capacity(market_dispatch, opening_cap):
    '''Takes in the power for market dispatch and opening capacity, returns the closing capacity.'''
    
    if market_dispatch < 0:
        closing_cap_cand = opening_cap - market_dispatch * (CHARGE_EFF/100)
        return (max(0, min(round(closing_cap_cand), BATTERY_CAP)))
    elif market_dispatch > 0:
        closing_cap_cand = opening_cap - market_dispatch * (100/DISCHARGE_EFF)
        return (max(0, min(round(closing_cap_cand), BATTERY_CAP)))
    else:
        closing_cap_cand = opening_cap - market_dispatch * (100/DISCHARGE_EFF)
        return (max(0, min(round(closing_cap_cand), BATTERY_CAP)))

In [36]:
# Read in raw dataset provided

df = pd.read_excel("../../data/market_data.xlsx")

In [37]:
# Filter dataframe to Victoria's spot prices only and convert
# "Time (UTC+10) column to datetime type"

vic_spotprice = df.filter(items=['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)'])
vic_spotprice['Time (UTC+10)'] = pd.to_datetime(vic_spotprice['Time (UTC+10)'])

In [38]:
# Slice dataframe to 17/7/2020 only for checkpoint task
checkpoint3 = vic_spotprice.loc[vic_spotprice['Time (UTC+10)'].dt.date == date(2020,7,17)]
checkpoint3['Time (UTC+10)'] = checkpoint3['Time (UTC+10)'].dt.time
copy = checkpoint3

In [39]:
# Sort dataframe in terms of spot prices from lowest to highest
sort_prices = copy.sort_values(by=['Regions VIC Trading Price ($/MWh)', 'Time (UTC+10)'], ascending=True)

# Take the five lowest spot prices to determine charging periods
five_lowest = sort_prices[0:5]

five_lowest_sorted = five_lowest.sort_values(by=['Time (UTC+10)'])
five_lowest_sorted

,Time (UTC+10),Regions VIC Trading Price ($/MWh)
44571,13:30:00,46.23
44572,14:00:00,39.75
44573,14:30:00,39.94
44574,15:00:00,39.97
44575,15:30:00,42.72


In [40]:
# Take four periods with highest spot prices during the day to determine
# discharging periods
four_highest = sort_prices.tail(4)
four_highest_sorted = four_highest.sort_values(by=['Time (UTC+10)'])

# Ensure that period options for charging are before options for discharging
charging_options = sort_prices[sort_prices['Time (UTC+10)'] < four_highest_sorted['Time (UTC+10)'].iloc[0]]
five_lowest_new = charging_options.head(5)

In [41]:
# Create new columns to be able to calculate total market revenue

copy["Charge Forecast"] = 0
copy["Discharge Forecast"] = 0
copy["Raw Power"] = 0
copy["Market Dispatch"] = 0
copy["Market Revenue"] = 0
copy["Opening Capacity"] = 0
copy["Closing Capacity"] = 0

In [42]:
# Set charge and discharge forecasts in dataframe
for i in range(len(copy)):
    if (copy.iloc[i])['Time (UTC+10)'] in list(five_lowest_new['Time (UTC+10)']):
        copy["Charge Forecast"].values[i] = 1
        
    elif (copy.iloc[i])['Time (UTC+10)'] in list(four_highest['Time (UTC+10)']):
        copy["Discharge Forecast"].values[i] = 1

In [43]:
for i in range(len(copy)):
    row = copy.iloc[i]
    
    # ith period opening capacity = (i-1)th period closing capacity
    if i != 0:
        copy["Opening Capacity"].values[i] = copy["Closing Capacity"].values[i-1]
    
    # Use formulas to calculate corresponding cell values
    copy["Raw Power"].values[i] = raw_power(row["Charge Forecast"], row["Discharge Forecast"], copy["Opening Capacity"].values[i])
    copy["Market Dispatch"].values[i] = market_dispatch(copy["Raw Power"].values[i])
    copy["Market Revenue"].values[i] = market_revenue(copy["Market Dispatch"].values[i], copy["Regions VIC Trading Price ($/MWh)"].values[i])
    copy["Closing Capacity"].values[i] = closing_capacity(copy["Market Dispatch"].values[i], copy["Opening Capacity"].values[i])

In [44]:
copy

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Charge Forecast,Discharge Forecast,Raw Power,Market Dispatch,Market Revenue,Opening Capacity,Closing Capacity
44544,00:00:00,79.09,0,0,0,0,0,0,0
44545,00:30:00,75.51,0,0,0,0,0,0,0
44546,01:00:00,73.98,0,0,0,0,0,0,0
44547,01:30:00,75.57,0,0,0,0,0,0,0
44548,02:00:00,71.94,0,0,0,0,0,0,0
44549,02:30:00,74.10,0,0,0,0,0,0,0
44550,03:00:00,67.36,0,0,0,0,0,0,0
44551,03:30:00,58.04,1,0,-300,-150,-8785,0,135
44552,04:00:00,51.85,1,0,-300,-150,-7848,135,270
44553,04:30:00,74.53,0,0,0,0,0,270,270


In [45]:
print(sum(copy["Market Revenue"]))

33537
